In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
def lstsqk(x, y):
    return np.multiply(x, y).mean(axis = 0) / np.multiply(x, x).mean(axis = 0)

In [ ]:
def lstsqkerr(x, y):
    return 1 / np.sqrt(len(x)) * np.sqrt(np.multiply(y, y).mean(axis = 0) / np.multiply(x, x).mean(axis = 0) - lstsqk(x,y) ** 2)

# First

In [ ]:
df1 = pd.read_excel('experimental_data.xlsx', sheet_name = 'Data').reset_index(drop = True)

In [ ]:
t0 = (-1) * 273.15
Re = 10 # Ohm

In [ ]:
df1['T, K'] = df1['T, C'] - t0
df1['I, А'] = df1['U1, В'] /  Re
df1['Q, Вт'] = df1['U2, В'] * df1['I, А']
df1['R, Ом'] = df1['U2, В'] / df1['I, А']

In [ ]:
tset = set(df1['T, C'])
df_list = []
tlist = list(tset)
tlist.sort()

for i in tlist:
    df_list.append(df1.loc[df1['T, C'] == i])

In [ ]:
tlistk = [x - t0 for x in tlist]

# General Data

In [ ]:
parr1 = []
earr1 = []
for d in df_list:
    p, v = np.polyfit(d['Q, Вт'], d['R, Ом'], deg = 1, cov = True)
    parr1.append(p)
    earr1.append(np.sqrt(np.diag(v)))

In [ ]:
def calculate_least_squares(df):
    df1 = df[:].copy()
    
    df1['Q * R'] = df1['Q, Вт'] * df1['R, Ом']
    
    df1['Q^2'] = df1['Q, Вт'] * df1['Q, Вт']
    df1['R^2'] = df1['R, Ом'] * df1['R, Ом']
    
    x_m = df1['Q, Вт'].mean()
    y_m = df1['R, Ом'].mean()
    
    xx_m = df1['Q^2'].mean()
    xy_m = df1['Q * R'].mean()
    yy_m = df1['R^2'].mean()
    
    return df1, x_m, y_m, xx_m, xy_m, yy_m

In [ ]:
mnkdata = []
for i in range(len(df_list)):
    df, _, _, _, _, _ = calculate_least_squares(df_list[i])
    mnkdata.append(df)

In [ ]:
for i in range(len(mnkdata)):
    mnkdata[i].to_excel('plots/output{}.xlsx'.format(i + 1))

In [ ]:
df_list[0]

In [ ]:
# Calculation for T = 24.80C
df1, x_m, y_m, xx_m, xy_m, yy_m = calculate_least_squares(df_list[0])
print("Q_mean = {:.4f}".format(x_m))
print("R_mean = {:.4f}".format(y_m))
print("Q2_mean = {:.4f}".format(xx_m))
print("R2_mean = {:.4f}".format(yy_m))
print("RQ_mean = {:.4f}".format(xy_m))

In [ ]:
(xy_m - x_m * y_m) / (xx_m - x_m ** 2)

# Plot Load Curves

In [ ]:
def get_min_max(series, k1 = 3 / 4, k2 = 11 / 10):
    return k1 * min(series), k2 * max(series)

In [ ]:
def plot_load_curve(ax, df, p, e, i):
    y_min, y_max = get_min_max(df['R, Ом'], k1 = 10 / 11)
    x_min, x_max = get_min_max(df['Q, Вт'])
    
    ax.xaxis.set_major_locator(plt.MultipleLocator(0.025))
    ax.yaxis.set_major_locator(plt.MultipleLocator(0.5))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(0.025))
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.5))
    
    ax.set_ylim(bottom = y_min, top = y_max)
    ax.set_xlim(left = x_min, right = x_max)
    
    ax.grid(visible = True, linestyle = '--', which = 'both')
    x = np.linspace(x_min, x_max, 1000)
    ax.scatter(df['Q, Вт'], df['R, Ом'])
    ax.plot(x, p[0] * x + p[1], label = "Линейная модель: $R(Q) = a + b \cdot Q$")
    ax.plot([], [], ' ', label = '$b = {k:.3f} \pm {dk:.3f}, a = {C:.3f} \pm {dC:.3f}$'.format(k = p[0], dk = e[0], C = p[1], dC = e[1]))
    
    ax.legend()
    ax.set_xlabel('$Q, Вт$')
    ax.set_ylabel('$R, Ом$')

In [ ]:
for x in range(0, len(tset), 4):
    fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize=(11.7, 8.3))
    ax = ax.reshape(4)
    fig.set_tight_layout(True)
    fig.suptitle('Рис. {}. Зависимость R(Q) от мощности Q'.format((x + 4) // 4))
    for xi in range(2 * 2):
        if (x + xi >= len(list(tset))):
            break
        ax[xi].set_title('Рис. {}.{}. $T = {:.2f} C$'.format((x + 4) // 4, xi + 1, tlist[x + xi]))
        plot_load_curve(ax[xi], df_list[x + xi], parr1[x + xi], earr1[x + xi], x + xi)
    fig.savefig('plots/plot{}.pdf'.format((x // 4) + 1))

In [ ]:
rarr = [x[1] for x in parr1]

In [ ]:
p1, v1 = np.polyfit(tlistk, rarr, deg = 1, cov = True)
err1 = np.sqrt(np.diag(v1))

In [ ]:
p1

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. {}. Зависимость R(T)'.format(3))

y_min, y_max = get_min_max(rarr, k1 = 9 / 10, k2 = 11 / 10)
x_min, x_max = get_min_max(tlistk, k1 = 100 / 101, k2 = 102 / 101)

ax.xaxis.set_major_locator(plt.MultipleLocator(5))
ax.yaxis.set_major_locator(plt.MultipleLocator(0.5))
ax.xaxis.set_minor_locator(plt.MultipleLocator(2.5))
ax.yaxis.set_minor_locator(plt.MultipleLocator(0.25))

ax.set_ylim(bottom = y_min, top = y_max)
ax.set_xlim(left = x_min, right = x_max)

ax.grid(visible = True, linestyle = '--', which = 'both')
x = np.linspace(x_min, x_max, 1000)
ax.scatter(tlistk, rarr)

ax.plot(x, p1[0] * x + p1[1], label = "Линейная модель: $R(T) = a + b \cdot T$")
ax.plot([], [], ' ', label = '$b = {k:.4f} \pm {dk:.4f}, a = {C:.4f} \pm {dC:.4f}$'.format(k = p1[0], dk = err1[0], C = p1[1], dC = err1[1]))

ax.legend()
ax.set_xlabel('$T, К$')
ax.set_ylabel('$R, Ом$')

fig.savefig('plots/plot{}.pdf'.format(3))

In [ ]:
alpha = 1 / (p1[0] * 273.0 + p1[1]) * p1[0]

In [ ]:
p1[0] * 273 + p1[1]

In [ ]:
L = 0.370
ln21 = 5.3

In [ ]:
xiarr = []
for i in range(len(df_list)):
    x = 1 / parr1[i][0] * p1[0] * 1 / (2 * np.pi * L) * ln21
    xiarr.append(x)
    print("T = {:.1f}, xi = {:.4f}".format(tlist[i], x))

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. {}. Зависимость $\kappa(T)$'.format(4))

y_min, y_max = get_min_max(xiarr, k1 = 9 / 10, k2 = 11 / 10)
x_min, x_max = get_min_max(tlistk, k1 = 100 / 101, k2 = 102 / 101)

ax.xaxis.set_major_locator(plt.MultipleLocator(5))
ax.yaxis.set_major_locator(plt.MultipleLocator(1e-3))
ax.xaxis.set_minor_locator(plt.MultipleLocator(2.5))
ax.yaxis.set_minor_locator(plt.MultipleLocator(5e-4))

ax.set_ylim(bottom = y_min, top = y_max)
ax.set_xlim(left = x_min, right = x_max)

ax.grid(visible = True, linestyle = '--', which = 'both')
x = np.linspace(x_min, x_max, 1000)
ax.plot(tlistk, xiarr, label = '$\kappa(T)$', marker = 'd')

ax.legend()
ax.set_xlabel('$T, К$')
ax.set_ylabel('$\kappa$ $Вт / м \cdot К$')

fig.savefig('plots/plot{}.pdf'.format(4))

In [ ]:
lntlistk = np.log(np.array(tlistk))
lnxiarr = np.log(np.array(xiarr))

In [ ]:
p2, v2 = np.polyfit(lntlistk, lnxiarr, deg = 1, cov = True)
err2 = np.sqrt(np.diag(v2))

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. {}. Зависимость $ln(\kappa(ln(T)))$'.format(5))

y_min, y_max = get_min_max(lnxiarr, k1 = 9 / 10, k2 = 11 / 10)
x_min, x_max = get_min_max(lntlistk, k1 = 100 / 101, k2 = 102 / 101)

ax.xaxis.set_major_locator(plt.MultipleLocator(5e-2))
ax.yaxis.set_major_locator(plt.MultipleLocator(1e-1))
ax.xaxis.set_minor_locator(plt.MultipleLocator(2.5e-2))
ax.yaxis.set_minor_locator(plt.MultipleLocator(5e-2))

ax.set_ylim(bottom = y_max, top = y_min)
ax.set_xlim(left = x_min, right = x_max)

ax.grid(visible = True, linestyle = '--', which = 'both')
x = np.linspace(x_min, x_max, 1000)
ax.scatter(lntlistk, lnxiarr)

ax.plot(x, p2[0] * x + p2[1], label = "Линейная модель: $R(T) = a + b \cdot T$")
ax.plot([], [], ' ', label = '$b = {k:.4f} \pm {dk:.4f}, a = {C:.4f} \pm {dC:.4f}$'.format(k = p2[0], dk = err2[0], C = p2[1], dC = err2[1]))


ax.legend()
ax.set_xlabel('$ln(T)$')
ax.set_ylabel('$ln(\kappa)$')

fig.savefig('plots/plot{}.pdf'.format(5))